In [7]:
import pandas as pd
import numpy as np
import glob
import re
from collections import Counter
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import tree
import random

In [13]:
cc = pd.read_csv('Data/Ready_for_ML/lnab_playoff.csv')
cc.set_index(['Series_ID'],inplace=True)

In [19]:
y = cc['Winner'].values
X = cc.drop(['Opponent 0', 'Winner', 'Year', 'Opponent 1'], axis=1)
X = X.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
clfn = MLPClassifier(solver='lbfgs', activation='logistic', hidden_layer_sizes=(50,), early_stopping=True, alpha=0.01)
clfn.fit(X_train,y_train)

clft = tree.DecisionTreeClassifier()
clft.fit(X_train, y_train)



print(roc_auc_score(y_test, clfn.predict(X_test)))
print(clfn.predict_proba(X_test))
print(clfn.predict(X_test))
print(y_test)

print(roc_auc_score(y_test, clft.predict(X_test)))
print(clft.predict(X_test))
print(y_test)

0.633333333333
[[ 0.35519888  0.64480112]
 [ 0.52272721  0.47727279]
 [ 0.53433555  0.46566445]
 [ 0.66474527  0.33525473]
 [ 0.64422419  0.35577581]
 [ 0.42988033  0.57011967]
 [ 0.44489354  0.55510646]
 [ 0.39124285  0.60875715]
 [ 0.37890086  0.62109914]
 [ 0.54956786  0.45043214]
 [ 0.4878221   0.5121779 ]]
[1 0 0 0 0 1 1 1 1 0 1]
[1 0 0 1 0 0 1 0 1 1 1]
0.466666666667
[0 1 0 1 1 0 0 0 1 0 0]
[1 0 0 1 0 0 1 0 1 1 1]
